In [1]:
import pandas as pd
import json 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [3]:
rf=RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)

In [4]:
def rolling_averages(group, cols, new_cols):
    group=group.sort_values("date")
    rolling_stats=group[cols].rolling(3,closed='left').mean()
    group[new_cols]=rolling_stats
    group=group.dropna(subset=new_cols)
    return group

In [97]:
def TrainModel(data):
    matches=pd.read_csv('matches.csv',index_col=0)
    
    matches=matches.groupby("team").get_group(data["team"])
    new_row=pd.DataFrame(data,index=[0])
    matches = pd.concat([matches,new_row], ignore_index = True)
    print(matches.index)
    
    matches["date"]=pd.to_datetime(matches["date"])
    matches["opp_code"]=matches["opponent"].astype("category").cat.codes
    matches["hour"]=matches["time"].str.replace(":.+","",regex=True).astype("int")
    matches["formation_code"]=matches["formation"].str.replace(r'\D',"",regex=True).astype("int")
    matches["day_code"]=matches["date"].dt.dayofweek
    matches["venue_code"]=matches["venue"].astype("category").cat.codes
    matches["target"]=(matches["result"]=="W").astype("int")
    cols=["gf","ga","sh","sot","dist","fk","pk","pkatt"]
    
    predictors=["venue_code","opp_code","hour","day_code",'formation_code']
    
    new_cols=[f"{c}_rolling" for c in cols]
    
    
    
    matches_rolling=rolling_averages(matches,cols,new_cols)
    last_index=matches_rolling.index.sort_values()[-1]
    # print("LATS INDEX=",matches_rolling.index[-1])
    
    prev_matches=matches_rolling.loc[:last_index-1]
    
    train=prev_matches
    rf.fit(train[predictors+new_cols],train['target'])
    return rf, matches_rolling.loc[last_index:]

In [91]:
data = {
		"team":"Manchester City",
		"opponent":"Tottenham",
		"date":"2022-04-23",
		"time":"18:00",
		"comp":"Premier League",
		"round":"Matchweek 35",
		"day":"Sun",
		"venue":"Away",
		"result":"",
		"formation":"4-3-3"
	}

In [60]:
matches=pd.read_csv('matches.csv',index_col=0)

In [61]:
matches=matches.groupby("team").get_group(data["team"])

In [62]:
# matches=matches.drop([71])

In [63]:
new_row=pd.DataFrame(data,index=[0])
matches = pd.concat([matches,new_row], ignore_index = True)
matches.tail()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
67,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Chelsea,...,Match Report,NaN,15.0,3.0,19.0,0.0,0.0,1.0,2021.0,Manchester City
68,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4.0,3.0,Newcastle Utd,...,Match Report,NaN,15.0,6.0,19.0,1.0,0.0,0.0,2021.0,Manchester City
69,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2.0,3.0,Brighton,...,Match Report,NaN,8.0,4.0,13.2,1.0,0.0,0.0,2021.0,Manchester City
70,2021-05-23,16:00,Premier League,Matchweek 38,Sun,Home,W,5.0,0.0,Everton,...,Match Report,NaN,21.0,11.0,15.3,0.0,0.0,0.0,2021.0,Manchester City
71,2022-04-23,18:00,Premier League,Matchweek 35,Sun,Away,,NaN,NaN,Tottenham,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Manchester City


In [64]:
matches["date"]=pd.to_datetime(matches["date"])
matches["opp_code"]=matches["opponent"].astype("category").cat.codes
matches["hour"]=matches["time"].str.replace(":.+","",regex=True).astype("int")
matches["formation_code"]=matches["formation"].str.replace(r'\D',"",regex=True).astype("int")
matches["day_code"]=matches["date"].dt.dayofweek
matches["venue_code"]=matches["venue"].astype("category").cat.codes
matches["target"]=(matches["result"]=="W").astype("int")
cols=["gf","ga","sh","sot","dist","fk","pk","pkatt"]

predictors=["venue_code","opp_code","hour","day_code",'formation_code']

new_cols=[f"{c}_rolling" for c in cols]

In [65]:
matches.tail()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,opp_code,hour,formation_code,day_code,venue_code,target
67,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1.0,2.0,Chelsea,...,0.0,1.0,2021.0,Manchester City,5,17,352,5,1,0
68,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4.0,3.0,Newcastle Utd,...,0.0,0.0,2021.0,Manchester City,13,20,433,4,0,1
69,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2.0,3.0,Brighton,...,0.0,0.0,2021.0,Manchester City,3,19,433,1,0,0
70,2021-05-23,16:00,Premier League,Matchweek 38,Sun,Home,W,5.0,0.0,Everton,...,0.0,0.0,2021.0,Manchester City,7,16,433,6,1,1
71,2022-04-23,18:00,Premier League,Matchweek 35,Sun,Away,,NaN,NaN,Tottenham,...,NaN,NaN,NaN,Manchester City,17,18,433,5,0,0


In [66]:
matches_rolling=rolling_averages(matches,cols,new_cols)
matches_rolling["venue_code"].tail()

29    0
30    1
31    1
32    1
71    0
Name: venue_code, dtype: int8

In [77]:
len(matches_rolling.index)

69

In [68]:
last_index=matches_rolling.index.sort_values()[-1]
last_index

71

In [84]:
matches_rolling.loc[:last_index-1]

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,venue_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
36,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,...,1,1,2.000000,2.333333,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333
37,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,...,0,0,1.333333,2.000000,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000
38,2020-10-31,12:30,Premier League,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,...,0,1,1.000000,0.666667,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000
39,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,...,1,0,1.000000,0.333333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000
40,2020-11-21,17:30,Premier League,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,...,0,0,1.000000,0.666667,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333
41,2020-11-28,15:00,Premier League,Matchweek 10,Sat,Home,W,5.0,0.0,Burnley,...,1,1,0.666667,1.000000,14.333333,5.000000,17.800000,0.666667,0.000000,0.333333
42,2020-12-05,15:00,Premier League,Matchweek 11,Sat,Home,W,2.0,0.0,Fulham,...,1,1,2.000000,1.000000,15.333333,4.666667,16.966667,0.000000,0.000000,0.333333
43,2020-12-12,17:30,Premier League,Matchweek 12,Sat,Away,D,0.0,0.0,Manchester Utd,...,0,0,2.333333,0.666667,18.333333,5.666667,15.800000,0.000000,0.333333,0.333333
44,2020-12-15,20:00,Premier League,Matchweek 13,Tue,Home,D,1.0,1.0,West Brom,...,1,0,2.333333,0.000000,14.333333,4.666667,16.400000,0.333333,0.333333,0.333333
45,2020-12-19,15:00,Premier League,Matchweek 14,Sat,Away,W,1.0,0.0,Southampton,...,0,1,1.000000,0.333333,16.666667,4.666667,16.666667,1.000000,0.333333,0.333333


In [85]:
prev_matches=matches_rolling.loc[:last_index-1]
    
train=prev_matches
rf.fit(train[predictors+new_cols],train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [88]:
matches_rolling.loc[last_index:]

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,venue_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
71,2022-04-23,18:00,Premier League,Matchweek 35,Sun,Away,,NaN,NaN,Tottenham,...,0,0,3.333333,1.0,16.0,5.333333,16.733333,0.333333,0.333333,0.333333


In [89]:
rf.predict(matches_rolling.loc[last_index:][predictors+new_cols])

array([1])

In [98]:
model,test_data=TrainModel(data)

cols=["gf","ga","sh","sot","dist","fk","pk","pkatt"]
new_cols=[f"{c}_rolling" for c in cols]
predictors=["venue_code","opp_code","hour","day_code",'formation_code']


result=model.predict(test_data[predictors+new_cols])

print("result=",result)

RangeIndex(start=0, stop=72, step=1)
result= [1]
